In [1]:
#import dependencies
import pandas as pd
import json
import os

In [2]:
jsondata = os.path.join("Resources","purchase_data.json")

In [3]:
#Read JSON data into a variable
with open(jsondata) as json_data:
    d = json.load(json_data)

In [4]:
#turn data into dataframe
game_df = pd.DataFrame(d, columns=['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'])
game_df.head()

,SN,Age,Gender,Item ID,Item Name,Price
0,Aelalis34,38,Male,165,Bone Crushing Silver Skewer,3.37
1,Eolo46,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32
2,Assastnya25,34,Male,174,Primitive Blade,2.46
3,Pheusrical25,21,Male,92,Final Critic,1.36
4,Aela59,23,Male,63,Stormfury Mace,1.27


In [19]:
#Player Count
total_players = len(game_df['SN'].value_counts())
Total_Players = pd.DataFrame({"Total Players": total_players}, index=[0])
Total_Players

,Total Players
0,573


In [6]:
#Purchasing Analysis

In [7]:
#Number of Unique Items
unique_items = len(game_df['Item ID'].value_counts())

#Average Purchase Price
average_price = game_df['Price'].mean()

#Total Number of Purchases
total_purchases = game_df['Item Name'].count()


#Total Revenue
total_revenue = game_df['Price'].sum()

#Create DataFrame
purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items],
                                   "Average Price": [average_price],
                                   "Total Purchases": [total_purchases],
                                   "Total Revenue": [total_revenue],
                                
})

#Reorder DataFrame
purchasing_analysis = purchasing_analysis[["Number of Unique Items", "Average Price","Total Purchases", "Total Revenue"]]
                                

#improve formatting
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${0:,.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${0:,.2f}".format)

purchasing_analysis

,Number of Unique Items,Average Price,Total Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [8]:
#Gender Demographics

In [86]:
#Group data by Gender and filter duplicates
grouped_df = game_df.groupby(["Gender"])
gender_df = grouped_gender.nunique()

#Total Gender
total_gender = gender_df["SN"].sum()

#Percentage and Count of Players
count = gender_df["SN"].unique()
percentage = gender_df["SN"]/ total_gender

#Create new dataframe
final_gender = pd.DataFrame({"Percentage of Players": percentage,
                            "Count":count})
#Change percentage format
final_gender["Percentage of Players"] = final_gender["Percentage of Players"].map("{:,.2%}".format) 

#Print final dataframe
final_gender


,Count,Percentage of Players
Gender,,
Female,100,17.45%
Male,465,81.15%
Other / Non-Disclosed,8,1.40%


In [101]:
#Purchasing Analysis (Gender)

#Purchase Count
purchase_count = game_df["Gender"].value_counts()

#Average Purchase Price

#Total Purchase Value
#Normalized Totals


TypeError: 'float' object is not subscriptable